# Chapter 03-04: 콜백 (Callbacks)

## 학습 목표
- 콜백의 개념과 동작 시점을 이해한다
- ModelCheckpoint, EarlyStopping, ReduceLROnPlateau 등 주요 내장 콜백을 활용할 수 있다
- TensorBoard 콜백으로 학습 과정을 시각화할 수 있다
- `tf.keras.callbacks.Callback`을 상속하여 커스텀 콜백을 구현할 수 있다

## 목차
1. [콜백이란?](#1.-콜백이란?)
2. [ModelCheckpoint](#2.-ModelCheckpoint)
3. [EarlyStopping](#3.-EarlyStopping)
4. [ReduceLROnPlateau](#4.-ReduceLROnPlateau)
5. [TensorBoard](#5.-TensorBoard)
6. [커스텀 콜백](#6.-커스텀-콜백)
7. [정리](#7.-정리)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

print("TensorFlow 버전:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

## 1. 콜백이란?

콜백(Callback)은 **학습 중 특정 시점에 자동으로 호출되는 함수(또는 객체)**이다.

### 콜백이 호출되는 시점

| 메서드 | 호출 시점 |
|--------|----------|
| `on_train_begin` | 전체 학습 시작 전 |
| `on_train_end` | 전체 학습 종료 후 |
| `on_epoch_begin` | 각 에포크 시작 전 |
| `on_epoch_end` | 각 에포크 종료 후 |
| `on_train_batch_begin` | 각 훈련 배치 시작 전 |
| `on_train_batch_end` | 각 훈련 배치 종료 후 |
| `on_test_begin` | 평가(evaluate) 시작 전 |
| `on_predict_begin` | 예측(predict) 시작 전 |

```python
# 콜백 사용 방법: model.fit의 callbacks 인수에 리스트로 전달
model.fit(
    X_train, y_train,
    epochs=100,
    callbacks=[callback1, callback2, callback3]  # 복수의 콜백을 리스트로 전달
)
```

In [ ]:
# ---------------------------------------------------
# 공통 데이터 및 모델 준비
# ---------------------------------------------------

# MNIST 데이터 로드
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(-1, 784).astype('float32') / 255.0
X_test  = X_test.reshape(-1, 784).astype('float32') / 255.0

# 빠른 실험용 서브셋
X_tr = X_train[:6000]
y_tr = y_train[:6000]
X_val = X_train[6000:8000]
y_val = y_train[6000:8000]

def build_model(lr=0.001):
    """실험용 간단한 MLP 모델 생성"""
    tf.random.set_seed(42)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("데이터 준비 완료")
print(f"  훈련: {X_tr.shape}, 검증: {X_val.shape}")

## 2. ModelCheckpoint

학습 중 가장 좋은 모델 또는 주기적으로 체크포인트를 자동 저장한다.

In [ ]:
# ---------------------------------------------------
# ModelCheckpoint: 최고 성능 모델 자동 저장
# ---------------------------------------------------

# 저장 경로 설정
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# ModelCheckpoint 콜백 설정
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'best_model.keras'),  # 저장 경로
    monitor='val_accuracy',   # 모니터할 지표
    save_best_only=True,      # True: 최고 성능 모델만 저장 (기본: 매 에포크)
    save_weights_only=False,  # False: 전체 모델 저장 (True: 가중치만)
    mode='max',               # 'max': 클수록 좋음 (accuracy), 'min': 작을수록 좋음 (loss)
    verbose=1                 # 저장 시 출력
)

# 에포크별 저장 (파일명에 에포크 번호와 지표 포함)
checkpoint_all = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_val_acc_{val_accuracy:.4f}.keras'),
    monitor='val_accuracy',
    save_best_only=False,     # 모든 에포크 저장
    verbose=0
)

model = build_model()
history = model.fit(
    X_tr, y_tr,
    epochs=5,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint_callback],
    verbose=1
)

print("\n=== 저장된 체크포인트 ===")
if os.path.exists(checkpoint_dir):
    for f in os.listdir(checkpoint_dir):
        print(f"  {f}")

# 저장된 최고 모델 로드
print("\n=== 저장된 최고 모델 로드 ===")
best_model_path = os.path.join(checkpoint_dir, 'best_model.keras')
if os.path.exists(best_model_path):
    loaded_model = tf.keras.models.load_model(best_model_path)
    test_loss, test_acc = loaded_model.evaluate(X_test, y_test, verbose=0)
    print(f"로드된 최고 모델 - 테스트 정확도: {test_acc:.4f}")

## 3. EarlyStopping

검증 성능이 개선되지 않을 때 학습을 조기 종료하여 과적합을 방지한다.

In [ ]:
# ---------------------------------------------------
# EarlyStopping: 조기 종료
# ---------------------------------------------------

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',           # 모니터할 지표
    patience=5,                   # 개선 없이 기다릴 에포크 수
    min_delta=0.001,              # 개선으로 인정할 최소 변화량
    restore_best_weights=True,    # 가장 좋은 가중치로 복원 (중요!)
    mode='min',                   # 'min': 손실이 감소해야 개선
    baseline=None,                # 기준값 (None: 없음)
    verbose=1
)

# ModelCheckpoint와 함께 사용 (권장 패턴)
checkpoint_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'early_stop_best.keras'),
    monitor='val_loss',
    save_best_only=True,
    verbose=0
)

model_es = build_model(lr=0.001)
print("EarlyStopping patience=5로 최대 30 에포크 학습 시도")
history_es = model_es.fit(
    X_tr, y_tr,
    epochs=30,                          # 최대 에포크 (조기 종료 가능)
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, checkpoint_best],
    verbose=0
)

actual_epochs = len(history_es.history['loss'])
print(f"\n실제 학습 에포크 수: {actual_epochs} / 30")
print(f"최고 검증 손실: {min(history_es.history['val_loss']):.4f}")
print(f"최종 검증 정확도: {history_es.history['val_accuracy'][-1]:.4f}")
print(f"restore_best_weights=True이므로 조기 종료 시점이 아닌 최고 성능 시점의 가중치 사용")

# 학습 곡선 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
epochs_range = range(1, actual_epochs + 1)

axes[0].plot(epochs_range, history_es.history['loss'], label='훈련 손실')
axes[0].plot(epochs_range, history_es.history['val_loss'], label='검증 손실')
axes[0].axvline(x=actual_epochs, color='red', linestyle='--', label='조기 종료 시점')
axes[0].set_xlabel('에포크')
axes[0].set_ylabel('손실')
axes[0].set_title('EarlyStopping - 손실 곡선')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(epochs_range, history_es.history['accuracy'], label='훈련 정확도')
axes[1].plot(epochs_range, history_es.history['val_accuracy'], label='검증 정확도')
axes[1].set_xlabel('에포크')
axes[1].set_ylabel('정확도')
axes[1].set_title('EarlyStopping - 정확도 곡선')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. ReduceLROnPlateau

In [ ]:
# ---------------------------------------------------
# ReduceLROnPlateau: 학습률 자동 감소
# ---------------------------------------------------

# 학습률을 기록하는 도우미 콜백
class LRRecorder(tf.keras.callbacks.Callback):
    """에포크별 학습률을 기록하는 보조 콜백"""
    def __init__(self):
        super().__init__()
        self.lr_history = []
    
    def on_epoch_end(self, epoch, logs=None):
        lr = float(self.model.optimizer.learning_rate)
        self.lr_history.append(lr)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',   # 모니터 지표
    factor=0.5,           # 학습률 감소 배율: new_lr = lr * factor
    patience=3,           # 개선 없이 기다릴 에포크
    min_lr=1e-6,          # 학습률 하한선
    min_delta=0.001,      # 최소 개선량
    cooldown=0,           # 학습률 감소 후 쉬어가는 에포크 수
    verbose=1
)

lr_recorder = LRRecorder()

# 의도적으로 큰 학습률로 시작하여 감소 과정 관찰
model_rlr = build_model(lr=0.01)
history_rlr = model_rlr.fit(
    X_tr, y_tr,
    epochs=20,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[reduce_lr, lr_recorder],
    verbose=0
)

# 학습률 변화 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
epochs_range = range(1, len(history_rlr.history['loss']) + 1)

axes[0].semilogy(epochs_range, lr_recorder.lr_history, 'bo-', linewidth=2)
axes[0].set_xlabel('에포크')
axes[0].set_ylabel('학습률 (로그 스케일)')
axes[0].set_title('ReduceLROnPlateau - 학습률 변화')
axes[0].grid(True, alpha=0.3)

axes[1].plot(epochs_range, history_rlr.history['val_loss'], 'r-', linewidth=2)
axes[1].set_xlabel('에포크')
axes[1].set_ylabel('검증 손실')
axes[1].set_title('검증 손실 변화')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"초기 학습률: {lr_recorder.lr_history[0]:.6f}")
print(f"최종 학습률: {lr_recorder.lr_history[-1]:.6f}")

## 5. TensorBoard

TensorBoard는 학습 과정을 실시간으로 시각화하는 강력한 도구이다.

In [ ]:
# ---------------------------------------------------
# TensorBoard 콜백 설정
# ---------------------------------------------------

# 타임스탬프를 포함한 로그 디렉토리 생성 (실험별 구분)
log_dir = os.path.join(
    './logs',
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,              # 로그 저장 경로
    histogram_freq=1,             # N 에포크마다 가중치/편향 히스토그램 기록 (0=비활성)
    write_graph=True,             # 모델 그래프 기록
    write_images=False,           # 가중치를 이미지로 기록 여부
    update_freq='epoch',          # 'epoch' 또는 'batch' 또는 정수
    profile_batch=0               # 성능 프로파일링 배치 (0=비활성)
)

model_tb = build_model()
print(f"TensorBoard 로그 경로: {log_dir}")
print()

model_tb.fit(
    X_tr, y_tr,
    epochs=5,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[tensorboard_callback],
    verbose=1
)

print("\n=== TensorBoard 실행 방법 ===")
print(f"터미널에서 다음 명령어를 실행:")
print(f"  tensorboard --logdir=./logs")
print(f"브라우저에서 http://localhost:6006 접속")
print()
print("Jupyter Notebook에서 직접 실행:")
print("  %load_ext tensorboard")
print("  %tensorboard --logdir ./logs")

## 6. 커스텀 콜백

`tf.keras.callbacks.Callback`을 상속하여 원하는 동작을 구현할 수 있다.

In [ ]:
# ---------------------------------------------------
# 커스텀 콜백: 에포크별 학습률 및 메트릭 출력
# ---------------------------------------------------

class DetailedLoggingCallback(tf.keras.callbacks.Callback):
    """에포크 종료 시 학습률과 주요 메트릭을 상세하게 출력하는 콜백"""
    
    def __init__(self, print_every=1):
        super().__init__()
        self.print_every = print_every  # N 에포크마다 출력
        self.lr_history = []           # 학습률 기록
        self.metrics_history = {}      # 메트릭 기록
    
    def on_train_begin(self, logs=None):
        """학습 시작 시 헤더 출력"""
        print("=" * 60)
        print("학습 시작")
        print(f"옵티마이저: {self.model.optimizer.__class__.__name__}")
        print(f"초기 학습률: {float(self.model.optimizer.learning_rate):.6f}")
        print("=" * 60)
    
    def on_epoch_begin(self, epoch, logs=None):
        """에포크 시작 전 현재 학습률 출력"""
        current_lr = float(self.model.optimizer.learning_rate)
        self.lr_history.append(current_lr)
    
    def on_epoch_end(self, epoch, logs=None):
        """에포크 종료 후 상세 로그 출력"""
        logs = logs or {}
        
        # 메트릭 기록
        for key, value in logs.items():
            if key not in self.metrics_history:
                self.metrics_history[key] = []
            self.metrics_history[key].append(value)
        
        # N 에포크마다 출력
        if (epoch + 1) % self.print_every == 0:
            current_lr = float(self.model.optimizer.learning_rate)
            loss = logs.get('loss', 0)
            val_loss = logs.get('val_loss', 0)
            acc = logs.get('accuracy', 0)
            val_acc = logs.get('val_accuracy', 0)
            
            print(f"[에포크 {epoch+1:3d}] "
                  f"lr={current_lr:.6f} | "
                  f"loss={loss:.4f} | "
                  f"val_loss={val_loss:.4f} | "
                  f"acc={acc:.4f} | "
                  f"val_acc={val_acc:.4f}")
    
    def on_train_end(self, logs=None):
        """학습 완료 후 요약 출력"""
        print("=" * 60)
        print("학습 완료")
        if 'val_accuracy' in self.metrics_history:
            best_epoch = np.argmax(self.metrics_history['val_accuracy']) + 1
            best_val_acc = max(self.metrics_history['val_accuracy'])
            print(f"최고 검증 정확도: {best_val_acc:.4f} (에포크 {best_epoch})")
        print("=" * 60)


# 커스텀 콜백 사용
detailed_logger = DetailedLoggingCallback(print_every=2)
reduce_lr2 = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=0
)

model_custom = build_model(lr=0.005)
model_custom.fit(
    X_tr, y_tr,
    epochs=10,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[detailed_logger, reduce_lr2],
    verbose=0  # 기본 출력 비활성화
)

## 7. 정리

### 콜백 사용 패턴 가이드

#### 기본 권장 조합
```python
callbacks = [
    # 최고 모델 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath='best_model.keras',
        monitor='val_loss',
        save_best_only=True
    ),
    # 과적합 방지 조기 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    # 학습률 자동 감소
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5
    ),
    # 시각화
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
```

### 콜백별 핵심 파라미터 요약

| 콜백 | 핵심 파라미터 | 설명 |
|------|--------------|------|
| ModelCheckpoint | `save_best_only`, `monitor` | 최고 모델만 저장 vs 모든 에포크 저장 |
| EarlyStopping | `patience`, `restore_best_weights` | 조기 종료 후 최고 가중치 복원 |
| ReduceLROnPlateau | `factor`, `patience`, `min_lr` | 학습률 감소 배율과 최소 하한선 |
| TensorBoard | `histogram_freq`, `log_dir` | 히스토그램 기록 주기 |

### 커스텀 콜백 구현 포인트
- 적절한 메서드(`on_epoch_end`, `on_batch_end` 등)를 오버라이드
- `self.model`로 현재 모델에 접근 가능
- `logs` 딕셔너리에서 현재 메트릭 값 참조 가능